In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
import random
import tensorflow.keras.utils as utils
import pydot
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from sklearn.ensemble import RandomForestRegressor
class Mics_Model:
    def __init__(self, dataset_dir, use_encoder=True, sampling_method="Vanilla", global_model="NN", group_number = 3):
        self.dataset_dir = dataset_dir
        self.use_encoder = use_encoder
        self.sampling_method = sampling_method
        self.global_model = global_model
        self.group_number = group_number
        self.raw_data = None
            
    #ASSUMPTION: column 0: index, column 1: labels, remaining columns are features. 
    def get_raw_data(self, index_col=0):
        raw_data = pd.read_csv(self.dataset_dir, index_col=0)
        raw_data = raw_data.fillna(raw_data.mean())
        raw_data = raw_data.sample(frac=1, random_state=41)
        self.raw_data = raw_data
        
    #This method assigns the feature number = column number - 1 (exclude label column). After that, it returns a list of
    #input feature numbers according to group count. Ex: for 28 cols, 27 features, 4 group_num: returns [7,7,6,7] 
    #Output of this function can be fed to get_model methods as inp_sizes input.
    def get_input_group_lenthgs(self):
        count = self.group_number
        input_sizes = [None]*count
        feature_num = len(self.raw_data.columns) - 1
        for i in range(count):
            group_size = round(feature_num/(count-i))
            input_sizes[i] = group_size
            feature_num = feature_num - group_size
        return input_sizes
    
    #This method returns grouped column numbers
    #[[1,4,5],[2,3,6]]
    def get_grouped_feature_cols(self):
        grouped_feature_cols = [None]*self.group_number
        feature_num = len(self.raw_data.columns) - 1
        inp_sizes = self.get_input_group_lenthgs()
        total_nums = [i for i in range(feature_num)]
        for j in range(len(inp_sizes)):
            size = inp_sizes[j]
            temp_list = random.sample(total_nums, size)
            grouped_feature_cols[j] = temp_list
            for k in temp_list:
                total_nums.remove(k)
        return grouped_feature_cols
    
    #groups is a list of lists [[1,4,5], [2,3,6]] which is output of get_grouped_feature_cols method
    #returns: [[train_x1, train_x2..., train_xn, train_y],
    #          [test_x1, test_x2..., test_xn, test_y]]
    def get_features_and_labels(self, groups):
        row_num = len(self.raw_data.index)
        
        trainx_df = self.raw_data.iloc[:int(0.8*row_num), 1:]
        trainy_df = self.raw_data.iloc[:int(0.8*row_num), 0]
        testx_df = self.raw_data.iloc[int(0.8*row_num):, 1:]
        testy_df = self.raw_data.iloc[int(0.8*row_num):, 0]        
        
        scaler = StandardScaler()
        trainx_scaled = pd.DataFrame(scaler.fit_transform(trainx_df), columns = trainx_df.columns, index = trainx_df.index)
        textx_scaled = pd.DataFrame(scaler.transform(testx_df), columns = testx_df.columns, index = testx_df.index)
        
        features_and_labels = [[None for _ in range(self.group_number + 1)] for _ in range(2)]
        
        for index, group in enumerate(groups):
            train_temp = trainx_scaled.iloc[:,group]
            features_and_labels[0][index] = train_temp.values
            test_temp = textx_scaled.iloc[:,group]
            features_and_labels[1][index] = test_temp.values            
        features_and_labels[0][self.group_number] = trainy_df.values
        features_and_labels[1][self.group_number] = testy_df.values   
        return features_and_labels
    
    #returns [[train_x1, train_x2..., train_xn, train_y],
    #         [test_x1, test_x2..., test_xn, test_y]]
    
    def get_vanilla_encoder_model(self, inp_size):
        inputs = keras.layers.Input(shape=(inp_size))
        h1 = keras.layers.Dense(10, activation="relu")(inputs)
        h1 = keras.layers.Dense(10, activation="relu")(inputs)        
        outputs = keras.layers.Dense(inp_size, activation="relu")(h1)
        return keras.Model(inputs,outputs)
    
    #This subclass is created for sampling for a given mean and log_variance.
    class Sampling(layers.Layer):
        def call(self, inputs):
            z_mean, z_log_var = inputs
            batch = tf.shape(z_mean)[0]
            dim = tf.shape(z_mean)[1]
            epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
            return z_mean + tf.exp(0.5 * z_log_var) * epsilon #multiplies with std
    
    def get_variatonal_encoder_model(self, inp_size):
        inputs = layers.Input(shape=(inp_size))
        h1 = layers.Dense(10, activation="relu")(inputs)
        z_mean = layers.Dense(inp_size, name="z_mean")(h1)
        z_log_var = layers.Dense(inp_size, name="z_log_var")(h1)
        outputs = self.Sampling()([z_mean, z_log_var])
        return keras.Model(inputs,outputs)
    #New sampling methods can be added here 
    
    def get_nn_model(self, inp_sizes, drop_out=0.25, hidden_num = 4, hidden_size=32, activation="relu"):
        inp_group_count = len(inp_sizes)
        inputs = [None]*inp_group_count
        for i in range(inp_group_count):
            inputs[i] = keras.layers.Input(shape=(inp_sizes[i]), name="input_"+str(i))
        if self.use_encoder == True:
            encoders = [None]*inp_group_count
            if self.sampling_method == "Vanilla":
                for j in range(inp_group_count):
                    encoders[j] = self.get_vanilla_encoder_model(inp_sizes[j])
            elif self.sampling_method == "Variational":
                for j in range(inp_group_count):
                    encoders[j] = self.get_variatonal_encoder_model(inp_sizes[j])
            #This place can be extended if new sampling methods are added.
            global_inputs = [None]*inp_group_count
            for k in range(inp_group_count):
                global_inputs[k] = encoders[k](inputs[k])
            global_input = keras.layers.concatenate(global_inputs)
        else:
            global_input = keras.layers.concatenate(inputs)
            
        h = keras.layers.Dense(hidden_size, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(global_input)
        h = keras.layers.Dropout(drop_out)(h)
        for hidden in range(hidden_num):
            h = keras.layers.Dense(hidden_size, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(h)
            h = keras.layers.Dropout(drop_out)(h) 

        outputs = keras.layers.Dense(1, activation=activation)(h)    
        return keras.Model(inputs=inputs, outputs = outputs) 
    
    def default_exp(self, batch_size = 300):
        inp_sizes = self.get_input_group_lenthgs()
        groups = self.get_grouped_feature_cols()
        features_and_labels = self.get_features_and_labels(groups)
        MICS_model = self.get_nn_model(inp_sizes=inp_sizes)
        callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50), 
                keras.callbacks.ReduceLROnPlateau("val_loss", factor = 0.8, patience=30,
                                                 verbose = 2, mode = "auto", 
                                                  min_lr = 1e-6)]
        MICS_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss=keras.losses.MeanSquaredError())
        history = MICS_model.fit(x = features_and_labels[0][:-1], y = features_and_labels[0][-1],  
                                 validation_data = (features_and_labels[1][:-1], features_and_labels[1][-1]),
                                 epochs=300, batch_size = batch_size, callbacks=callback)
        training_val_loss = history.history["val_loss"]
        best_row_index = np.argmin(training_val_loss)
        best_val_loss = training_val_loss[best_row_index]
        print(best_val_loss)
        
    def default_exp_house(self, batch_size = 300):
        inp_sizes = self.get_input_group_lenthgs()
        groups = self.get_grouped_feature_cols()
        features_and_labels = self.get_features_and_labels(groups)
        MICS_model = self.get_nn_model(inp_sizes=inp_sizes, activation="sigmoid")
        callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50), 
                keras.callbacks.ReduceLROnPlateau("val_loss", factor = 0.8, patience=30,
                                                 verbose = 2, mode = "auto", 
                                                  min_lr = 1e-6)]
        MICS_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss=keras.losses.BinaryCrossentropy(), metrics=["accuracy"])
        history = MICS_model.fit(x = features_and_labels[0][:-1], y = features_and_labels[0][-1],  
                                 validation_data = (features_and_labels[1][:-1], features_and_labels[1][-1]),
                                 epochs=300, batch_size = batch_size, callbacks=callback)
        training_val_loss = history.history["val_loss"]
        best_row_index = np.argmin(training_val_loss)
        best_val_loss = training_val_loss[best_row_index]
        
        training_acc = history.history["val_accuracy"]
        best_row_index_acc = np.argmax(training_acc)
        best_val_acc = training_acc[best_row_index_acc]        
        print("best val loss is: " + str(best_val_loss))
        print("best val accuracy is: " + str(best_val_acc))
        return best_val_acc
                


In [14]:
dataset_dir = "./Datasets/energydata_use.csv"
deneyelim = Mics_Model(dataset_dir, use_encoder=False, group_number=3)
deneyelim.get_raw_data()
deneyelim.default_exp()

Epoch 1/300
53/53 [==============================] - 0s 4ms/step - loss: 13109.8086 - val_loss: 9614.2871
Epoch 2/300
53/53 [==============================] - 0s 3ms/step - loss: 10177.5186 - val_loss: 8459.7764
Epoch 3/300
53/53 [==============================] - 0s 3ms/step - loss: 9770.9971 - val_loss: 8177.3047
Epoch 4/300
53/53 [==============================] - 0s 3ms/step - loss: 9751.5273 - val_loss: 8073.0054
Epoch 5/300
53/53 [==============================] - 0s 3ms/step - loss: 9428.1650 - val_loss: 8623.4980
Epoch 6/300
53/53 [==============================] - 0s 3ms/step - loss: 9315.1943 - val_loss: 8077.5474
Epoch 7/300
53/53 [==============================] - 0s 3ms/step - loss: 9261.9297 - val_loss: 7736.1147
Epoch 8/300
53/53 [==============================] - 0s 3ms/step - loss: 9189.3506 - val_loss: 8135.8569
Epoch 9/300
53/53 [==============================] - 0s 3ms/step - loss: 9148.8760 - val_loss: 7741.5586
Epoch 10/300
53/53 [==============================] -

KeyboardInterrupt: 

In [48]:
dataset_dir2 = "./Datasets/houseprices_ready.csv"
accuracies = []
for i in range(5):
    deneyelim2 = Mics_Model(dataset_dir2, use_encoder=True, sampling_method="Vanilla", group_number=5)
    deneyelim2.get_raw_data()
    acc = deneyelim2.default_exp_house(batch_size=32)

Epoch 1/300
37/37 [==============================] - 0s 6ms/step - loss: 0.6393 - accuracy: 0.7774 - val_loss: 0.3689 - val_accuracy: 0.9007
Epoch 2/300
37/37 [==============================] - 0s 2ms/step - loss: 0.4298 - accuracy: 0.8622 - val_loss: 0.3326 - val_accuracy: 0.8973
Epoch 3/300
37/37 [==============================] - 0s 3ms/step - loss: 0.3899 - accuracy: 0.8724 - val_loss: 0.3222 - val_accuracy: 0.8973
Epoch 4/300
37/37 [==============================] - 0s 2ms/step - loss: 0.3770 - accuracy: 0.8673 - val_loss: 0.3034 - val_accuracy: 0.8836
Epoch 5/300
37/37 [==============================] - 0s 2ms/step - loss: 0.3844 - accuracy: 0.8836 - val_loss: 0.3006 - val_accuracy: 0.8904
Epoch 6/300
37/37 [==============================] - 0s 2ms/step - loss: 0.3660 - accuracy: 0.8801 - val_loss: 0.2985 - val_accuracy: 0.9041
Epoch 7/300
37/37 [==============================] - 0s 2ms/step - loss: 0.3376 - accuracy: 0.8896 - val_loss: 0.3006 - val_accuracy: 0.8801
Epoch 8/300
3

In [30]:
deneyelim2 = Mics_Model(dataset_dir2, use_encoder=True, group_number=3)
deneyelim2.get_raw_data()
a = deneyelim2.default_exp_house(batch_size=32)

Epoch 1/300
37/37 [==============================] - 0s 6ms/step - loss: 0.6258 - accuracy: 0.7697 - val_loss: 0.3884 - val_accuracy: 0.8938
Epoch 2/300
37/37 [==============================] - 0s 3ms/step - loss: 0.4342 - accuracy: 0.8733 - val_loss: 0.3383 - val_accuracy: 0.9007
Epoch 3/300
37/37 [==============================] - 0s 2ms/step - loss: 0.3809 - accuracy: 0.8887 - val_loss: 0.3667 - val_accuracy: 0.8973
Epoch 4/300
37/37 [==============================] - 0s 2ms/step - loss: 0.3939 - accuracy: 0.8801 - val_loss: 0.3067 - val_accuracy: 0.8938
Epoch 5/300
37/37 [==============================] - 0s 2ms/step - loss: 0.3660 - accuracy: 0.8896 - val_loss: 0.2846 - val_accuracy: 0.8904
Epoch 6/300
37/37 [==============================] - 0s 2ms/step - loss: 0.3304 - accuracy: 0.8904 - val_loss: 0.2721 - val_accuracy: 0.8973
Epoch 7/300
37/37 [==============================] - 0s 2ms/step - loss: 0.3093 - accuracy: 0.9050 - val_loss: 0.2583 - val_accuracy: 0.9041
Epoch 8/300
3

In [32]:
a

In [134]:
r = deneyelim.get_features_and_labels(groups)

9

In [106]:
deneyelim.get_features_and_labels(groups)

[[                         RH_6        T4        T8    RH_out        T5  \
  date                                                                    
  2016-01-11 17:00:00  0.769623 -0.761611 -1.924584  0.786817 -1.385479   
  2016-01-11 17:10:00  0.762633 -0.761611 -1.924584  0.786817 -1.385479   
  2016-01-11 17:20:00  0.729854 -0.810232 -1.924584  0.786817 -1.385479   
  2016-01-11 17:30:00  0.739495 -0.834542 -1.983431  0.786817 -1.385479   
  2016-01-11 17:40:00  0.792640 -0.834542 -1.983431  0.786817 -1.359694   
  ...                       ...       ...       ...       ...       ...   
  2016-04-30 07:30:00 -0.294374 -0.569336 -0.106203  1.096424  0.110056   
  2016-04-30 07:40:00 -0.287143 -0.609117 -0.106203  1.070624  0.058486   
  2016-04-30 07:50:00 -0.281118 -0.569336 -0.159166  1.044823  0.110056   
  2016-04-30 08:00:00 -0.300159 -0.569336 -0.159166  1.019022  0.161626   
  2016-04-30 08:10:00 -0.329323 -0.569336 -0.159166  0.928720  0.257031   
  
                      

In [56]:
a = [None]*3
b = [a,a]


In [57]:
b

[[None, None, None], [None, None, None]]

In [58]:
b[0][1] = 5
b

[[None, 5, None], [None, 5, None]]

In [61]:
a=[[None]*3]*2
a

[[None, None, None], [None, None, None]]

In [62]:
a[0][2]=5
a

[[None, None, 5], [None, None, 5]]

In [63]:
x = [[None for _ in range(3)] for _ in range(2)]
x

[[None, None, None], [None, None, None]]

In [64]:
x[1][2]=2
x

[[None, None, None], [None, None, 2]]

In [107]:
type([1,2,3])

list

In [112]:
a = [1,4,5]
(a[:-1])

[1, 4]

In [115]:
c = [[2,4,6],[1,3,5]]
c[1][-1]

5